# Milestone 2
Annie Anand
Jacob Stambaugh

Check Covid data.

In [11]:
%%bigquery
SELECT * from covid_19_staging.Spring_Cases;

,fips,admin2,state,country,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,None,None,MS Zaandam,2020-04-12 23:17:00,None,None,9,2,0,7,MS Zaandam
1,NaN,None,None,MS Zaandam,2020-04-13 23:07:34,None,None,9,2,0,7,MS Zaandam
2,NaN,None,None,MS Zaandam,2020-04-14 23:33:12,None,None,9,2,0,7,MS Zaandam
3,NaN,None,None,MS Zaandam,2020-04-15 22:56:32,None,None,9,2,0,7,MS Zaandam
4,NaN,None,None,MS Zaandam,2020-04-16 23:30:31,None,None,9,2,0,7,MS Zaandam
...,...,...,...,...,...,...,...,...,...,...,...,...
182099,NaN,None,Falkland Islands (Islas Malvinas),United Kingdom,2020-04-05 23:06:26,-51.7963,-59.5236,2,0,0,2,"Falkland Islands (Islas Malvinas), United Kingdom"
182100,NaN,None,Falkland Islands (Islas Malvinas),United Kingdom,4/6/20 23:21,-51.7963,-59.5236,2,0,0,2,"Falkland Islands (Islas Malvinas), United Kingdom"
182101,NaN,None,Falkland Islands (Islas Malvinas),United Kingdom,4/4/20 23:34,-51.7963,-59.5236,1,0,0,1,"Falkland Islands (Islas Malvinas), United Kingdom"
182102,NaN,None,Falkland Islands (Islas Malvinas),United Kingdom,2020-04-05 23:06:26,-51.7963,-59.5236,2,0,0,2,"Falkland Islands (Islas Malvinas), United Kingdom"


## 3. Creat Tables from staging tables

Remove unused columns and update titles, and grab only U.S data

In [21]:
%%bigquery
CREATE TABLE datamart.T1 AS
SELECT state,admin2 as city, last_update, confirmed,deaths,recovered,active,concat(state,',',admin2) as combined_key
from covid_19_staging.Spring_Cases
where country='US' and admin2 is not NULL;

""


check table

In [22]:
%%bigquery
select * from datamart.T1

,state,city,last_update,confirmed,deaths,recovered,active,combined_key
0,Iowa,Ida,3/29/20 23:08,0,0,0,0,"Iowa,Ida"
1,Iowa,Ida,3/28/20 23:05,0,0,0,0,"Iowa,Ida"
2,Iowa,Ida,2020-03-26 23:48:35,0,0,0,0,"Iowa,Ida"
3,Iowa,Ida,2020-03-27 22:14:55,0,0,0,0,"Iowa,Ida"
4,Iowa,Ida,2020-03-25 23:33:19,0,0,0,0,"Iowa,Ida"
...,...,...,...,...,...,...,...,...
166297,Maryland,Prince George's,2020-04-20 23:44:33,3583,112,0,3471,"Maryland,Prince George's"
166298,New Jersey,Union,2020-04-18 22:32:47,8959,356,0,8603,"New Jersey,Union"
166299,New Jersey,Union,2020-04-18 22:32:47,8959,356,0,8603,"New Jersey,Union"
166300,California,Los Angeles,2020-04-20 23:36:47,13823,619,0,13204,"California,Los Angeles"


Create Day table by selecting combined_key, last_update

In [35]:
%%bigquery
CREATE TABLE datamart.Day_temp AS
SELECT combined_key,cast(last_update as string) as last_update, concat(combined_key,',',cast(last_update as string)) as day_key from datamart.T1;

""


Create cases table by selcting concat(combined_key,last_update),case data

In [36]:
%%bigquery
CREATE TABLE datamart.Cases_temp AS
SELECT combined_key, concat(combined_key,',', cast(last_update as string)) as day_key, confirmed, deaths, recovered from datamart.T1;

""


create location table by removing "extra"

In [37]:
%%bigquery
CREATE TABLE datamart.Location_temp AS
SELECT state,city, combined_key 
FROM datamart.T1
where city!='Ida';

""


### remove duplicates

In [38]:
%%bigquery
CREATE TABLE datamart.Location AS
SELECT distinct * from datamart.Location_temp;

""


In [39]:
%%bigquery
CREATE TABLE datamart.Cases AS
SELECT distinct * from datamart.Cases_temp;

""


In [40]:
%%bigquery
CREATE TABLE datamart.Day AS
SELECT distinct * from datamart.Day_temp;

""


### ACS 

In [1]:
%%bigquery
CREATE TABLE datamart.demographics AS
SELECT state, Total_Population, Male, Females, White, Black_or_African_American as Black, Asian, Hispanic_or_Latino as Hispanic, population_over_18, male_population_over_18, female_population_over_18 FROM acs_staging.demo_and_housing;

""


In [5]:
%%bigquery
CREATE TABLE datamart.economics AS
SELECT state,labor_force,Employed,Unemployed,Unemployment_Rate,drove_alone,carpooled,Public_transportation,Walked,Worked_from_home,Mean_travel_time_to_work,Service_occupations,Agriculture_forestry,Construction,Manufacturing,Retail_trade,Information,Self_employed,Median_family_income,Mean_family_income, Median_earnings_for_workers,health_insurance_coverage,No_health_insurance_coverage,families_below_poverty_level FROM acs_staging.economics;

""


In [6]:
%%bigquery
CREATE TABLE datamart.housing AS
SELECT state,Total_housing_units,Occupied_housing_units,Vacant_housing_units,Homeowner_vacancy_rate,Rental_vacancy_rate,one_unit_detached,one_unit_attached,units_2,units_3_or_4,units_5_9,units_10_to_19,twenty_or_more_units,Mobile_home,Built_2014_or_later,Built_2010_to_2013,Built_2000_to_2009,Built_1990_to_1999,Built_1980_to_1989,Built_1970_to_1979,Built_1960_to_1969,Built_1950_to_1959,Built_1940_to_1949,Built_1939_or_earlier,Median_rooms,No_bedroom,Owner_occupied,Renter_occupied,Utility_gas,Electricity,Fuel_oil_kerosene,Coal_or_coke,Wood,Solar_energy,Other_fuel,No_fuel_used,Lacking_complete_plumbing_facilities,Lacking_complete_kitchen_facilities,units_with_a_mortgage,units_paying_rent,median_rent FROM acs_staging.housing_characteristics;

""


In [10]:
%%bigquery
CREATE TABLE datamart.social AS
SELECT state,Total_households,Married_couple_family,Male_householder_no_spouse,Female_householder_no_spouse,Average_household_size,Average_family_size,Children,Other_relatives,Other_nonrelatives,Male_Never_married,Males_Separated,Males_Divorced,Females_Never_married,Females_Separated,Females_Divorced,Grandparents,Less_than_9th_grade_education,High_school_graduate,Some_college_education,Associate_degree,Bachelor_degree,Graduate_or_professional_degree,Different_house_in_the_US,Owns_house_Abroad,Born_in_US,Foreign_born,Naturalized_citizen,Not_a_us_citizen,language_at_home_english_only,language_at_home_other_than_english,language_at_home_spanish,owns_computer,has_broadband_internet_subscription FROM acs_staging.social_characteristics;

""


## 4. For each field in tables primitive data type

Casting was done when creating tables before hand! 

## 6. Contiue to explore data using SQL queries

Q1 - Select top 10 states+city and their the overall covid case count, deaths, and median family income on a given day in the US where the number of cases in the state is less than the confirmed cases in Texas.

In [29]:
%%bigquery
SELECT L.state,L.city, sum(C.confirmed) as Num_Cases,sum(C.deaths) as num_deaths, econ.Median_family_income
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state 
GROUP BY L.state,L.city,econ.Median_family_income
HAVING Num_Cases < (SELECT sum(C.confirmed) as Num_Cases from datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key where L.state='Texas' group by L.state )
Order by Num_cases desc
limit 10;


,state,city,Num_Cases,num_deaths,Median_family_income
0,Michigan,Wayne,53068320,3099600,75703
1,New Jersey,Bergen,45657120,2042640,105705
2,California,Los Angeles,43593600,1445520,91377
3,New York,Rockland,36953760,934560,89475
4,New Jersey,Essex,33682080,1917120,105705
5,New Jersey,Hudson,32965440,1106880,105705
6,Florida,Miami-Dade,32098560,472080,71348
7,New Jersey,Union,28860000,896880,105705
8,Pennsylvania,Philadelphia,28792080,741120,81075
9,Louisiana,Orleans,26624880,1231440,65105


Q2 - Select  top 10 states and their median house rooms, and number of houses lacking complete plumbing and the maximum confirmed cases on a day in the US.

In [15]:
%%bigquery
SELECT L.state,max(C.confirmed) as num_cases, house.Median_rooms, house.Lacking_complete_plumbing_facilities
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.housing as house on house.state = L.state
WHERE L.country='US' and L.longitude<(select avg(longitude) from datamart.Location where state='Nevada')
group by L.state,house.Median_rooms, house.Lacking_complete_plumbing_facilities
having num_cases >0
order by num_cases desc
limit 10
;

,state,num_cases,Median_rooms,Lacking_complete_plumbing_facilities
0,California,13823,5.000000000,53199
1,Washington,5174,5.500000000,9970
2,Nevada,2998,5.100000000,6920
3,Idaho,598,5.800000000,2099
4,Oregon,520,5.300000000,8930
5,Hawaii,385,4.600000000,3365
6,Alaska,155,4.800000000,8132
7,Montana,7,5.500000000,2540


Q3 - Selec top 10 states in terms of greatest number of covid deaths in 1 day, selecting poverty level, and unemployment levels for those states

In [18]:
%%bigquery
SELECT L.state,max(C.deaths) as greatest_deaths_on_a_day, econ.families_below_poverty_level,econ.Unemployment_Rate
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state
group by L.state,econ.families_below_poverty_level,econ.Unemployment_Rate
ORDER BY greatest_deaths_on_a_day desc
limit 10
;

,state,greatest_deaths_on_a_day,families_below_poverty_level,Unemployment_Rate
0,New York,14604,0.090000000,4.400000000
1,Michigan,1148,0.090000000,5.000000000
2,Illinois,915,0.080000000,4.800000000
3,New Jersey,787,0.060000000,4.700000000
4,California,619,0.080000000,5.100000000
5,Connecticut,512,0.070000000,5.300000000
6,Massachusetts,372,0.060000000,3.900000000
7,Pennsylvania,370,0.080000000,4.500000000
8,Washington,346,0.060000000,4.600000000
9,Louisiana,339,0.140000000,5.500000000


Q4 - Select the top 10(in terms of civd deaths) states and their poverty level %, unenploymnet rate, and # with health insurance WHERE those states have less than number of people insured than the national average.

In [21]:
%%bigquery
SELECT L.state,max(C.confirmed) as greatest_cases_on_a_day, econ.families_below_poverty_level,econ.Unemployment_Rate,econ.health_insurance_coverage 
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state
group by L.state, econ.families_below_poverty_level,econ.Unemployment_Rate,econ.health_insurance_coverage 
HAVING econ.health_insurance_coverage < (select AVG(health_insurance_coverage) from datamart.economics)
order by greatest_cases_on_a_day desc
limit 10
;

,state,greatest_cases_on_a_day,families_below_poverty_level,Unemployment_Rate,health_insurance_coverage
0,Connecticut,8320,0.070000000,5.300000000,3307378
1,Louisiana,6148,0.140000000,5.500000000,4135437
2,Maryland,3583,0.060000000,4.500000000,5589015
3,Rhode Island,3491,0.070000000,4.200000000,1001033
4,Nevada,2998,0.090000000,5.100000000,2695788
5,District of Columbia,2927,0.090000000,6.300000000,672093
6,Missouri,2289,0.090000000,3.800000000,5416963
7,Wisconsin,2191,0.060000000,3.200000000,5422750
8,Colorado,1784,0.060000000,3.700000000,5210724
9,Utah,1671,0.060000000,3.200000000,2871538


Q5 Select top 10 states and their covid stats (case,deaths, and recovered), and race percentages, health insurace % of States WHERE the hispanic population is greater than national average.

In [34]:
%%bigquery
SELECT L.state,max(C.confirmed) as greatest_cases_on_a_day,max(C.deaths) as greatest_deaths_on_a_day, (demo.White/demo.Total_Population) as pct_white, (demo.Black/demo.Total_Population) as pct_black, (demo.Asian/demo.Total_Population) as pct_asian, (demo.Hispanic/demo.Total_Population) as pct_hispanic,econ.Unemployment_Rate,(econ.health_insurance_coverage/demo.Total_Population) as pct_with_health_insurance 
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state
group by L.state,econ.Unemployment_Rate,pct_with_health_insurance,pct_white,pct_black,pct_asian,pct_hispanic
HAVING pct_hispanic> (select AVG(demo.Hispanic/demo.Total_Population) FROM datamart.demographics as demo)
order by greatest_cases_on_a_day desc
limit 10
;

,state,greatest_cases_on_a_day,greatest_deaths_on_a_day,pct_white,pct_black,pct_asian,pct_hispanic,Unemployment_Rate,pct_with_health_insurance
0,New York,141235,14604,0.631950,0.158547,0.086371,0.192729,4.400000000,0.935884
1,Illinois,22101,915,0.713714,0.141275,0.056661,0.174900,4.800000000,0.912662
2,California,13823,619,0.594372,0.057758,0.148446,0.394179,5.100000000,0.910999
3,New Jersey,13011,787,0.671489,0.135534,0.096436,0.209053,4.700000000,0.910095
4,Florida,9657,223,0.745427,0.160215,0.027927,0.263698,4.500000000,0.855417
5,Connecticut,8320,512,0.746147,0.111280,0.046670,0.168557,5.300000000,0.927661
6,Texas,4977,78,0.733667,0.122566,0.049872,0.397465,4.400000000,0.802889
7,Rhode Island,3491,152,0.786974,0.073764,0.035022,0.162969,4.200000000,0.944940
8,Nevada,2998,137,0.645784,0.096159,0.084788,0.292387,5.100000000,0.875212
9,Arizona,2636,75,0.783354,0.047224,0.033209,0.317445,5.100000000,0.873404


## 7. Create data visuals

Veiw 1:

In [35]:
%%bigquery
CREATE VIEW reports.State_Racial_and_Unemployment_and_Health_Insurance AS
SELECT L.state,max(C.confirmed) as greatest_cases_on_a_day,max(C.deaths) as greatest_deaths_on_a_day, (demo.White/demo.Total_Population) as pct_white, (demo.Black/demo.Total_Population) as pct_black, (demo.Asian/demo.Total_Population) as pct_asian, (demo.Hispanic/demo.Total_Population) as pct_hispanic,econ.Unemployment_Rate,(econ.health_insurance_coverage/demo.Total_Population) as pct_with_health_insurance 
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.economics as econ on econ.state = L.state INNER JOIN datamart.demographics as demo on demo.state = L.state
group by L.state,econ.Unemployment_Rate,pct_with_health_insurance,pct_white,pct_black,pct_asian,pct_hispanic
HAVING pct_hispanic> (select AVG(demo.Hispanic/demo.Total_Population) FROM datamart.demographics as demo)
order by greatest_cases_on_a_day desc
limit 10
;

""


veiw 2:

In [36]:
%%bigquery
CREATE VIEW reports.State_max_cases_and_lacking_plumbing AS
SELECT L.state,max(C.confirmed) as num_cases, house.Median_rooms, house.Lacking_complete_plumbing_facilities
FROM datamart.Location as L INNER JOIN datamart.Day as D on L.combined_key=D.combined_key INNER JOIN datamart.Cases as C on C.day_key=D.day_key INNER JOIN datamart.housing as house on house.state = L.state
WHERE L.country='US' and L.longitude<(select avg(longitude) from datamart.Location where state='Nevada')
group by L.state,house.Median_rooms, house.Lacking_complete_plumbing_facilities
having num_cases >0
order by num_cases desc
limit 10
;

""
